In [ ]:
import requests
import json
import time
from datetime import datetime
import os

# Configuration
NOETL_SERVER = os.getenv('NOETL_SERVER', 'http://localhost:8082')
PLAYBOOK_PATH = 'tests/fixtures/playbooks/oauth/google_gcs'
PLAYBOOK_FILE = 'tests/fixtures/playbooks/oauth/google_gcs/google_gcs_oauth.yaml'

print(f"NoETL Server: {NOETL_SERVER}")
print(f"Playbook Path: {PLAYBOOK_PATH}")
print(f"Test Time: {datetime.now()}")

## Step 1: Register Playbook

Register the Google GCS OAuth playbook with the NoETL catalog.

In [ ]:
# Read playbook content
with open(PLAYBOOK_FILE, 'r') as f:
    playbook_content = f.read()

# Register playbook
response = requests.post(
    f"{NOETL_SERVER}/api/catalog/register",
    json={
        'path': PLAYBOOK_PATH,
        'content': playbook_content
    }
)

result = response.json()
print(f"Registration Status: {result.get('status')}")
print(f"Version: {result.get('version')}")

assert result.get('status') == 'success', f"Registration failed: {result}"

## Step 2: Execute Playbook

Run the playbook to test GCS OAuth authentication.

In [ ]:
# Execute playbook
response = requests.post(
    f"{NOETL_SERVER}/api/run/playbook",
    json={'path': PLAYBOOK_PATH}
)

result = response.json()
execution_id = result.get('execution_id')

print(f"Execution ID: {execution_id}")
assert execution_id is not None, f"Failed to start execution: {result}"

# Wait for execution
print("Waiting for execution to complete...")
time.sleep(20)

## Step 3: Query Execution Events

Retrieve all events for this execution to trace the flow.

In [ ]:
# Query events
query = f"""
SELECT node_name, event_type, created_at
FROM noetl.event
WHERE execution_id = {execution_id}
ORDER BY event_id
"""

response = requests.post(
    f"{NOETL_SERVER}/api/postgres/execute",
    json={'query': query, 'schema': 'noetl'}
)

events = response.json().get('result', [])
print(f"\nTotal Events: {len(events)}")
print("\nExecution Flow:")
for event in events:
    node_name, event_type, created_at = event
    print(f"  {node_name}: {event_type}")

## Validate Results

In [ ]:
# Check GCS API result
query = f"""
SELECT result->'status', result->'data'->'status_code'
FROM noetl.event
WHERE execution_id = {execution_id}
  AND node_name = 'list_buckets'
  AND event_type = 'action_completed'
"""

response = requests.post(
    f"{NOETL_SERVER}/api/postgres/execute",
    json={'query': query, 'schema': 'noetl'}
)

result = response.json().get('result', [])
if result:
    status = result[0][0]
    code = result[0][1]
    print(f"GCS API Status: {status}")
    print(f"HTTP Code: {code}")
    
    if code == 200:
        print("\n✅ Test PASSED - GCS OAuth working")
    else:
        print(f"\n⚠️ Test completed with HTTP {code}")
else:
    print("No result found")